# 2.1開始
使用pdfplumber套件讀取PL檔案名為"PSE_件號-X_PL"~"PSE_件號_MPPL"的pdf檔，框出有出現關鍵資訊(如件號、名稱、材質、規範)的地方，並轉換成txt檔

In [5]:
import pdfplumber
from PIL import Image, ImageDraw
import re
import os

# 指定pdf文件的資料夾路徑和保存txt文件的資料夾路徑
pdf_folder_path = r'-X_PL\pdf_file'
save_path = r'-X_PL\pdf2txt'

# 確認保存路徑存在
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 定義所有可能的命名模式
patterns = ['_PL', '_MPL', '_MPPL']

# 遍歷PDF文件夾中的所有pdf文件
for filename in os.listdir(pdf_folder_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, filename)
        with pdfplumber.open(pdf_path) as pdf:
            # 提取文件名中的核心部分
            match = None
            for pattern in patterns:
                match = re.search(r'PSE_(.*?)' + re.escape(pattern), filename)
                if match:
                    txt_filename = match.group(1)
                    break
            
            if not match:
                txt_filename = os.path.splitext(filename)[0]
            
            # 清理可能包含的多餘的下劃線和其他字元，並保留特定的字符
            txt_filename = re.sub(r'[^a-zA-Z0-9\-#G]', '', txt_filename)

            txt_filepath = os.path.join(save_path, txt_filename + ".txt")

            # 清空txt文件的內容
            with open(txt_filepath, 'w', encoding='utf-8') as txt_file:
                txt_file.write("")
            # 清空txt文件的內容
            with open(txt_filepath, 'w', encoding='utf-8') as txt_file:
                txt_file.write("")

            # 這裡放置你的文本區域擷取邏輯和寫入txt文件的代碼
            for page_number, page in enumerate(pdf.pages, start=1):
                #"US"的範圍
                top_u = 20
                bottom_u = 30
                left_u = 185
                right_u = 200
                us_text_within_range = page.crop((left_u, top_u, right_u, bottom_u)).extract_text()

                #"Boeing"的範圍
                top_b = 0
                bottom_b = 10
                left_b = 215
                right_b = 245
                boeing_text_within_range = page.crop((left_b, top_b, right_b, bottom_b)).extract_text()
                im = page.to_image().original.convert("RGB")
                draw = ImageDraw.Draw(im)

                # 繪製第一個邏輯的框選範圍
                draw.rectangle([left_u, top_u, right_u, bottom_u], outline="red", width=1)
                draw.rectangle([left_b, top_b, right_b, bottom_b], outline="red", width=1)
                #im.show()

                if us_text_within_range and "US" in us_text_within_range:
                    # 第二個邏輯和第三個邏輯只在第一頁運行
                    if page_number == 1:
                        # 第二個邏輯: 檢查是否存在 "NUMBER"
                        top1 = 55
                        bottom1 = 63
                        left1 = 480
                        right1 = 520
                        text_within_range1 = page.crop((left1, top1, right1, bottom1)).extract_text()

                        if text_within_range1 and "N U M B E R" in text_within_range1:
                            top1_1 = 66
                            bottom1_1 = 72
                            left1_1 = 480
                            right1_1 = 655
                            goal_within_range1 = page.crop((left1_1, top1_1, right1_1, bottom1_1)).extract_text()
                            if goal_within_range1:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("NUMBER:\n" + goal_within_range1 + '\n')
                                print(f"Page {page_number}: Saved NUMBER text to {txt_filepath}")

                        # 第三個邏輯: 擷取名稱
                        top2 = 80
                        bottom2 = 87
                        left2 = 52
                        right2 = 100
                        title_within_range = page.crop((left2, top2, right2, bottom2)).extract_text()

                        if title_within_range and "L IS T T IT L E" in title_within_range:
                            top2_1 = 90
                            bottom2_1 = 100
                            left2_1 = 55
                            right2_1 = 475
                            final_title_range = page.crop((left2_1, top2_1, right2_1, bottom2_1)).extract_text()
                            if final_title_range:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("TITLE:\n" + final_title_range + '\n')
                                print(f"Page {page_number}: Saved TITLE text to {txt_filepath}")

                        # 繪製第二個邏輯的框選範圍
                        draw.rectangle([left1, top1, right1, bottom1], outline="red", width=2)
                        draw.rectangle([left1_1, top1_1, right1_1, bottom1_1], outline="green", width=2)

                        # 繪製第三個邏輯的框選範圍
                        draw.rectangle([left2, top2, right2, bottom2], outline="blue", width=2)
                        draw.rectangle([left2_1, top2_1, right2_1, bottom2_1], outline="yellow", width=2)

                        # 顯示圖像
                        #im.show()
                    
                    # 第四個邏輯: 檢查是否存在 "QTY" 和 "PART OR IDENTIFYING"，在每頁運行
                    top3 = 110
                    bottom3 = 550
                    left3 = 50
                    right3 = 210
                    text_within_range3 = page.crop((left3, top3, right3, bottom3)).extract_text()

                    if text_within_range3 and "QTY" in text_within_range3 and "PART OR IDENTIFYING" in text_within_range3:
                        # 找到 "QTY" 的行號，動態設置 top3_1
                        lines = text_within_range3.split('\n')
                        for i, line in enumerate(lines):
                            if "QTY" in line:
                                top3_1 = top3 + (i * 10)  # 假設每行文字高度為 10
                                break

                        bottom3_1 = 550
                        left3_1 = 50
                        right3_1 = 330
                        goal_within_range3 = page.crop((left3_1, top3_1, right3_1, bottom3_1)).extract_text()
                        if goal_within_range3:
                            with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                txt_file.write("DESCRIPTION:\n" + goal_within_range3 + '\n')
                            print(f"Page {page_number}: Saved DESCRIPTION text to {txt_filepath}")

                        # 繪製第四個邏輯的框選範圍
                        draw.rectangle([left3, top3, right3, bottom3], outline="purple", width=2)
                        draw.rectangle([left3_1, top3_1, right3_1, bottom3_1], outline="orange", width=2)
                elif boeing_text_within_range and "Boeing" in boeing_text_within_range:
                    # 第二個邏輯和第三個邏輯只在第一頁運行
                    if page_number == 1:
                        # 第二個邏輯: 檢查是否存在 "NUMBER"
                        top1 = 40
                        bottom1 = 52
                        left1 = 475
                        right1 = 650
                        text_within_range1 = page.crop((left1, top1, right1, bottom1)).extract_text()

                        if text_within_range1 and "N U M B E R" in text_within_range1:
                            top1_1 = 58
                            bottom1_1 = 70
                            left1_1 = 475
                            right1_1 = 650
                            goal_within_range1 = page.crop((left1_1, top1_1, right1_1, bottom1_1)).extract_text()
                            if goal_within_range1:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("NUMBER:\n" + goal_within_range1 + '\n')
                                print(f"Page {page_number}: Saved NUMBER text to {txt_filepath}")

                        # 第三個邏輯: 擷取名稱
                        top2 = 75
                        bottom2 = 80
                        left2 = 50
                        right2 = 100
                        title_within_range = page.crop((left2, top2, right2, bottom2)).extract_text()

                        if title_within_range and "L IS T T IT L E" in title_within_range:
                            top2_1 = 84
                            bottom2_1 = 100
                            left2_1 = 50
                            right2_1 = 450
                            final_title_range = page.crop((left2_1, top2_1, right2_1, bottom2_1)).extract_text()
                            if final_title_range:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("TITLE:\n" + final_title_range + '\n')
                                print(f"Page {page_number}: Saved TITLE text to {txt_filepath}")

                        # 繪製第二個邏輯的框選範圍
                        draw.rectangle([left1, top1, right1, bottom1], outline="red", width=2)
                        draw.rectangle([left1_1, top1_1, right1_1, bottom1_1], outline="green", width=2)

                        # 繪製第三個邏輯的框選範圍
                        draw.rectangle([left2, top2, right2, bottom2], outline="blue", width=2)
                        draw.rectangle([left2_1, top2_1, right2_1, bottom2_1], outline="yellow", width=2)

                        # 顯示圖像
                        #im.show()
                    
                    # 第四個邏輯: 檢查是否存在 "QTY" 和 "PART OR IDENTIFYING"，在每頁運行
                    top3 = 100
                    bottom3 = 500
                    left3 = 50
                    right3 = 220
                    text_within_range3 = page.crop((left3, top3, right3, bottom3)).extract_text()

                    if text_within_range3 and "QTY" in text_within_range3 and "PART OR IDENTIFYING" in text_within_range3:
                        # 找到 "QTY" 的行號，動態設置 top3_1
                        lines = text_within_range3.split('\n')
                        for i, line in enumerate(lines):
                            if "QTY" in line:
                                top3_1 = top3 + (i * 10)  # 假設每行文字高度為 10
                                break

                        bottom3_1 = 550
                        left3_1 = 50
                        right3_1 = 335
                        goal_within_range3 = page.crop((left3_1, top3_1, right3_1, bottom3_1)).extract_text()
                        if goal_within_range3:
                            with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                txt_file.write("DESCRIPTION:\n" + goal_within_range3 + '\n')
                            print(f"Page {page_number}: Saved DESCRIPTION text to {txt_filepath}")

                        # 繪製第四個邏輯的框選範圍
                        draw.rectangle([left3, top3, right3, bottom3], outline="purple", width=2)
                        draw.rectangle([left3_1, top3_1, right3_1, bottom3_1], outline="orange", width=2)
                else:
                    # 第二個邏輯和第三個邏輯只在第一頁運行
                    if page_number == 1:
                        # 第二個邏輯: 檢查是否存在 "NUMBER"
                        top1 = 30
                        bottom1 = 37
                        left1 = 490
                        right1 = 530
                        text_within_range1 = page.crop((left1, top1, right1, bottom1)).extract_text()

                        if text_within_range1 and "N U M B E R" in text_within_range1:
                            top1_1 = 40
                            bottom1_1 = 45
                            left1_1 = 490
                            right1_1 = 680
                            goal_within_range1 = page.crop((left1_1, top1_1, right1_1, bottom1_1)).extract_text()
                            if goal_within_range1:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("NUMBER:\n" + goal_within_range1 + '\n')
                                print(f"Page {page_number}: Saved NUMBER text to {txt_filepath}")

                        # 第三個邏輯: 擷取名稱
                        top2 = 58
                        bottom2 = 65
                        left2 = 20
                        right2 = 70
                        title_within_range = page.crop((left2, top2, right2, bottom2)).extract_text()

                        if title_within_range and "L IS T T IT L E" in title_within_range:
                            top2_1 = 75
                            bottom2_1 = 80
                            left2_1 = 20
                            right2_1 = 480
                            final_title_range = page.crop((left2_1, top2_1, right2_1, bottom2_1)).extract_text()
                            if final_title_range:
                                with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                    txt_file.write("TITLE:\n" + final_title_range + '\n')
                                print(f"Page {page_number}: Saved TITLE text to {txt_filepath}")

                        # 繪製第二個邏輯的框選範圍
                        draw.rectangle([left1, top1, right1, bottom1], outline="red", width=2)
                        draw.rectangle([left1_1, top1_1, right1_1, bottom1_1], outline="green", width=2)

                        # 繪製第三個邏輯的框選範圍
                        draw.rectangle([left2, top2, right2, bottom2], outline="blue", width=2)
                        draw.rectangle([left2_1, top2_1, right2_1, bottom2_1], outline="yellow", width=2)

                        # 顯示圖像
                        #im.show()
                    
                    # 第四個邏輯: 檢查是否存在 "QTY" 和 "PART OR IDENTIFYING"，在每頁運行
                    top3 = 85
                    bottom3 = 550
                    left3 = 15
                    right3 = 200
                    text_within_range3 = page.crop((left3, top3, right3, bottom3)).extract_text()

                    if text_within_range3 and "QTY" in text_within_range3 and "PART OR IDENTIFYING" in text_within_range3:
                        # 找到 "QTY" 的行號，動態設置 top3_1
                        lines = text_within_range3.split('\n')
                        for i, line in enumerate(lines):
                            if "QTY" in line:
                                top3_1 = top3 + (i * 10)  # 假設每行文字高度為 10
                                break

                        bottom3_1 = 550
                        left3_1 = 15
                        right3_1 = 320
                        goal_within_range3 = page.crop((left3_1, top3_1, right3_1, bottom3_1)).extract_text()
                        if goal_within_range3:
                            with open(txt_filepath, 'a', encoding='utf-8') as txt_file:
                                txt_file.write("DESCRIPTION:\n" + goal_within_range3 + '\n')
                            print(f"Page {page_number}: Saved DESCRIPTION text to {txt_filepath}")

                        # 繪製第四個邏輯的框選範圍
                        draw.rectangle([left3, top3, right3, bottom3], outline="purple", width=2)
                        draw.rectangle([left3_1, top3_1, right3_1, bottom3_1], outline="orange", width=2)



Page 1: Saved NUMBER text to -X_PL\pdf2txt\113W6740-1.txt
Page 1: Saved TITLE text to -X_PL\pdf2txt\113W6740-1.txt
Page 2: Saved DESCRIPTION text to -X_PL\pdf2txt\113W6740-1.txt
Page 1: Saved NUMBER text to -X_PL\pdf2txt\284U2708-31.txt
Page 1: Saved TITLE text to -X_PL\pdf2txt\284U2708-31.txt
Page 2: Saved DESCRIPTION text to -X_PL\pdf2txt\284U2708-31.txt
Page 1: Saved NUMBER text to -X_PL\pdf2txt\284W1325-9.txt
Page 1: Saved TITLE text to -X_PL\pdf2txt\284W1325-9.txt
Page 2: Saved DESCRIPTION text to -X_PL\pdf2txt\284W1325-9.txt
Page 3: Saved DESCRIPTION text to -X_PL\pdf2txt\284W1325-9.txt
Page 1: Saved NUMBER text to -X_PL\pdf2txt\284W1336-13.txt
Page 1: Saved TITLE text to -X_PL\pdf2txt\284W1336-13.txt
Page 2: Saved DESCRIPTION text to -X_PL\pdf2txt\284W1336-13.txt
Page 3: Saved DESCRIPTION text to -X_PL\pdf2txt\284W1336-13.txt
Page 4: Saved DESCRIPTION text to -X_PL\pdf2txt\284W1336-13.txt
Page 1: Saved NUMBER text to -X_PL\pdf2txt\414W2525-131.txt
Page 1: Saved TITLE text to -X_

# 2.1結束

# 2.2開始
對轉換成txt檔的文件進行關鍵字(如件號、名稱、材質、規範)擷取並填入進去excel中

In [6]:
import os
import openpyxl
from openpyxl.styles import Alignment, PatternFill, Font

# 關鍵資訊擷取
def read_txt(filename):
    global pl, list_title, stock
    pl = []
    list_title = []
    stock = []
    accumulating_title = False
    current_title = []

    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 處理前3行，提取NUMBER和TITLE
    for content in lines[:3]:
        if "NUMBER" in content:
            pl.append(lines[1].strip())
        if "TITLE" in content:
            list_title.append(lines[3].strip())

    # 處理剩餘的行
    for content in lines[5:]:
        word = content.split()
        if not word:
            continue
        first_char_is_digit = word[0].isdigit() and len(word[0]) <3

        # 判斷是否開始累積標題
        if accumulating_title:
            if first_char_is_digit and 'CONDITION' not in content:
                list_title.append(' '.join(current_title).strip())
                current_title = []
                accumulating_title = False
            elif word[0] == 'AS':
                list_title.append(' '.join(current_title).strip())
                current_title = []
                accumulating_title = False
            elif content.startswith('(') or word[0] == 'PARENTHESIS' or word[0] == '*':
                list_title.append(' '.join(current_title).strip())
                current_title = []
                accumulating_title = False
            else:
                current_title.append(content.strip())
                continue

        # 條件：第一個單詞是數字且不包含 'CONDITION'
        if first_char_is_digit and 'CONDITION' not in content:
            index = word.index(word[0])
            pl_ans = ' '.join(word[index + 1: index+2]).strip()
            pl.append(pl_ans)
            current_title.append(' '.join(word[index + 2:]).strip())
            accumulating_title = True

        # 條件：第一個單詞是 'AS'
        elif word[0] == 'AS':
            index = word.index('REQD')
            pl_ans = ' '.join(word[index + 1: index+2]).strip()
            title_ans = ' '.join(word[index + 2:]).strip()
            pl.append(pl_ans)
            current_title.append(title_ans)
            accumulating_title = True

    # 處理最後一個標題
    if accumulating_title and current_title:
        list_title.append(' '.join(current_title).strip())

    print(pl)
    print(list_title)

# 將擷取到的關鍵資訊依照MBOM表格式匯入EXCEL中
def creat_excel(output_folder, file_name):
    global stock, pl, list_title 

    workbook = openpyxl.Workbook()
    sheet = workbook.active
    #將A欄填入NO
    sheet.cell(row=1, column=2).value = 'NO'

    #將B~J欄填入0~8
    for col in range(3, 12):
        sheet.cell(row=1, column=col).value = col - 3
    
    # 將 pl[] 的值存入 excel 的 K 欄
    sheet.cell(row=1, column=12).value = '件號\nPart Number'
    for i in range(0, len(pl)):
        sheet.cell(row=i+2, column=12).value = pl[i]



    # 將 list_title[] 的值存入 excel 的 L 欄
    sheet.cell(row = 1, column = 13).value = '名稱\nNOMENCLA TURE'
    for i in range(0, len(list_title)):
        sheet.cell(row=i+2, column=13).value = list_title[i]
    
    # 將 M 欄的標題設置為 '類別\nCode'
    sheet.cell(row = 1, column = 14).value = '類別\nCode'
    
    # 將 N 欄的標題設置為 '零件代碼\nShape'
    sheet.cell(row = 1, column = 15).value = '零件代碼\nShape'

    # 將 stock[] 的值存入 excel 的 O 欄
    sheet.cell(row = 1, column = 16).value = '材質\nMaterial'
    #for i in range(0, len(stock)):
        #sheet.cell(row = i+2, column = 15).value = stock[i]

    # 將 per[] 的值存入 excel 的 P 欄
    sheet.cell(row = 1, column = 17).value = '規範\nSpec.'
    #for i in range(0, len(per)):
        #sheet.cell(row = i+2, column = 16).value = per[i]
    
    #EXCEL排版設定
    target_row = 1
    last_col = sheet.max_column

    # 設置Alignment物件，指定水平置中和自動換行
    alignment = Alignment(horizontal='center', wrapText=True)

    # 遍歷指定行的每一列，並將文字設置為置中和自動換行
    for col in range(1, last_col + 1):
        cell = sheet.cell(row=target_row, column=col)
        cell.alignment = alignment
    
    #設定顏色與字形
    for col_index in range(1, last_col + 1):
        cell = sheet.cell(row=1, column=col_index)
        cell.fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
    #設定字形
        if isinstance(cell.value, str) and any('\u4e00' <= char <= '\u9fff' for char in cell.value):
                font = Font(name='Malgun Gothic', color='000000', bold=False)  # 設定要的中文字體
        else:
            font = Font(name='Calibri', color='000000', bold=False)  # 設定要的英文字體
                
        cell.font = font
    
    # 設定欄寬
    columns_to_adjust = ['L', 'M', 'P', 'Q']
    default_column_width = 30
    columns_to_adjust_1 = ['N', 'O']
    default_column_width_1 = 10

    for col_letter in columns_to_adjust:
        sheet.column_dimensions[col_letter].width = default_column_width
    for col_letter in columns_to_adjust_1:
        sheet.column_dimensions[col_letter].width = default_column_width_1

    workbook.save(os.path.join(output_folder, f"{file_name}_output.xlsx"))


# 輸入目標資料夾路徑給 creat_excel 函式
def read_folder(input_folder, output_folder):
    global index
    index = 1
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.txt'):
            file_path = os.path.join(input_folder, file_name)
            read_txt(file_path)
            base_name = os.path.splitext(file_name)[0]
            creat_excel(output_folder, base_name)
            index += 1

# 修改這行以指定目標資料夾和輸出資料夾路徑
input_folder_path = r"-X_PL\pdf2txt"
output_folder_path = r"-X_PL\final_output"
read_folder(input_folder_path, output_folder_path)

['113W6740-1', 'N0039948']
['SEAL, COVELIP DOOR, WING TE', 'MISCELLANEOUS, MOLDED PART']
['284U2708-31', '284U2706-20', '284U2706-25', '284U2708-22', '284U2708-30']
['DOOR ASSY - P715', 'HINGE ASSY - CTR', 'HINGE ASSY', 'DOOR', 'DOOR']
['284W1325-9', '284W1803-17', '284W1804-2', '284W1822-1', '284W1920-1', '284W1920-15', 'BACF3F005X015NN', 'BACG20Z-B000300', 'BACJ40D12-16', 'BACN10GH3-4', 'BACN10TL3-6', 'BACN10TL3-8', 'BACN10YR3CD', 'BACS53B1ES1', 'NAS1149D0332J']
['RACK ASSY - LOWER', 'BRACKET ASSY', 'BRACKET ASSY', 'BEAM', 'SHELF SUPPORT', 'HIRF SUPPORT', 'FILLER', 'ATN GROMMET BACG20ZB000300', 'BONDING JUMPER', 'NUTSPACER', 'NUTPLATE', 'NUT, SPACER PLATE, LIGHT DUTY', 'NUT', 'ELECTRICAL GROUND STUD', 'WASHER']
['284W1336-13', '284W1338-18', '284W1338-6', '284W1338-8', '284W1338-9', '284W1339-20', '284W1339-21', '284W1339-5', '284W1339-6', '284W1339-8', 'BACB30LH3-6', 'BACB30NT3K3', 'BACN10GH3A2', 'BACN10JN3CD', 'BACN10YR3CD', 'CB4001G3AA16H', 'CB4022G3CR8', 'NAS1149D0316J', 'NAS1149

# 2.2結束
# 2.3開始

將所有輸出的excel整合起來並依照MBOM表格式填入

In [7]:
import pandas as pd
import glob
import os

def combine_excel_files(folder_path, output_file):
    # 要合併的Excel檔案路徑
    folder_path = os.path.join(folder_path, '*.xlsx')  # 使用資料夾路徑並匹配所有xlsx檔案

    # 讀取所有Excel檔案
    all_files = glob.glob(folder_path)

    # 創建一個空的DataFrame用於存儲合併後的資料
    combined_data = pd.DataFrame()

    # 迭代讀取每個Excel檔案並合併到combined_data中
    for file in all_files:
        df = pd.read_excel(file)  # 讀取Excel檔案
        combined_data = pd.concat([combined_data, df], ignore_index=True)  # 將資料合併到combined_data中

    # 寫入合併後的資料到新的Excel檔案中
    combined_data.to_excel(output_file, index=False)  # 檔案名稱為output_file，不包括索引

# 使用範例
input_folder = r'-X_PL\final_output'  # 替換成你的資料夾路徑
output_file = r'-X_PL\final_output\combine.xlsx'  # 合併後的Excel檔案名稱

combine_excel_files(input_folder, output_file)

In [8]:
import pandas as pd
import glob
import os
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, PatternFill

def mbom_excel_files(folder_path, output_file):
    # 要合併的Excel檔案路徑
    folder_path = os.path.join(folder_path, '*.xlsx')  # 使用資料夾路徑並匹配所有xlsx檔案

    # 讀取所有Excel檔案
    all_files = glob.glob(folder_path)

    # 創建一個空的DataFrame用於存儲合併後的資料
    combined_data = pd.DataFrame()

    # 迭代讀取每個Excel檔案並合併到combined_data中
    for file in all_files:
        df = pd.read_excel(file)  # 讀取Excel檔案
        combined_data = pd.concat([combined_data, df], ignore_index=True)  # 將資料合併到combined_data中

    # 寫入合併後的資料到新的Excel檔案中
    combined_data.to_excel(output_file, index=False)  # 檔案名稱為output_file，不包括索引

    # 排版設定
    wb = load_workbook(output_file)
    sheet = wb.active
    target_row = 1
    last_col = sheet.max_column

    sheet.cell(row=1,column=1).value = ' '
    #將B欄填入NO
    sheet.cell(row=1, column=2).value = 'NO'
    # 設置Alignment物件，指定水平置中和自動換行
    alignment = Alignment(horizontal='center', wrapText=True)

    # 遍歷指定行的每一列，並將文字設置為置中和自動換行
    for col in range(1, last_col + 1):
        cell = sheet.cell(row=target_row, column=col)
        cell.alignment = alignment
    
    # 設定顏色與字形
    for col_index in range(1, last_col + 1):
        cell = sheet.cell(row=1, column=col_index)
        cell.fill = PatternFill(start_color="CCFFCC", end_color="CCFFCC", fill_type="solid")
        # 設定字形
        if isinstance(cell.value, str) and any('\u4e00' <= char <= '\u9fff' for char in cell.value):
            font = Font(name='Malgun Gothic', color='000000', bold=False)  # 設定要的中文字體
        else:
            font = Font(name='Calibri', color='000000', bold=False)  # 設定要的英文字體
        cell.font = font
    
    # 設定欄寬
    columns_to_adjust = ['L', 'M', 'P', 'Q']
    default_column_width = 30
    columns_to_adjust_1 = ['N', 'O']
    default_column_width_1 = 10

    for col_letter in columns_to_adjust:
        sheet.column_dimensions[col_letter].width = default_column_width
    for col_letter in columns_to_adjust_1:
        sheet.column_dimensions[col_letter].width = default_column_width_1

    # 儲存修改後的 Excel 檔案
    wb.save(output_file)

# 使用範例
input_folder = r'-X_PL\final_output'  # 替換成你的資料夾路徑
output_file = r'-X_PL\final_output\combine.xlsx'  # 合併後的Excel檔案名稱

mbom_excel_files(input_folder, output_file)


# 2.3結束